# Sportsbook Evaluation
This notebook will pull historical line data and game outcomes from covers.com and create a calibration curve which we can use to evaluate how effective sportsbooks are at setting betting lines for nhl games. We will then use this data to determine the accuracy of our own model that we create.

The code in this notebook will accomplish the following:
1. Get a list of dates that NHL games were played in the 2018/2019 season
2. For every game in the 2018/2019 season we will pull the betting lines and result of the game. We will use this season as it was the last full season played that wasn't shortened due to COVID.
3. Determine how accurate sportsbooks were at predicting the winner of games over the whole season through the use of a calibration curve.

In [4]:
from bs4 import BeautifulSoup
import datetime as dt
import matplotlib.pyplot as plt
import os
import pickle
from sklearn.calibration import calibration_curve
from sklearn.metrics import accuracy_score, brier_score_loss
import requests
import re
import sys
%matplotlib inline


## Remove Duplicates
This function will come in handy later. All it does is take a list and removes any duplicate values from that list. Example [cat,dog,duck,dog] will return [cat,dog,duck].

In [3]:
def remove_duplicates(x):
    """
    takes a list and removes duplicates from that list
    ...
    Parameters
    ----------
    x: list
        list from which duplicates will be removed
    Returns
    -------
    list
        list with duplicates removed
    """
    return list(dict.fromkeys(x))

## Pull NHL Betting Line Data
We will get our data from covers.com and hockey-reference.com. 

First we want a list of every date an NHL regular season game was played in the season specified. The list will contain datetime objects which each datetime object representing a date on which at least 1 nhl regular season game was played.

We will use the requests library in order to get this data. We will first send a get request to hockey-reference and receive a response. A response code of 200 means the request was successful. A response code outside the 200-400 range such as 404 means there was an error.

In [20]:
#we want data for the 2018/2019 season
year1 = 2018
year2 = 2019
url = f'https://www.hockey-reference.com/leagues/NHL_{year2}_games.html'

#get response from hockey-reference
resp = requests.get(url)
print(resp.status_code)

200


Lets print out the actual content of the response. The actual content is large so we need to open it in a separate text editor.

In [22]:
print(resp.text)


<!DOCTYPE html>
<html data-version="klecko-" data-root="/home/hr/build" itemscope itemtype="https://schema.org/WebSite" lang="en" class="no-js" >
<head>
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />
    <link rel="dns-prefetch" href="https://d2p3bygnnzw9w3.cloudfront.net/req/202201141" />
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script type="text/javascript" async=true>
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://quantcast.mgr.consensu.org'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, '/choice.js')
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);

By examining the text file we can see that starting in line 706 dates on which nhl games were played are shown in the site html. We need to extract all these dates.

In [25]:
# Find all the days games were played for year1 and year 2 and put them in a list
days1 = re.findall(f'html">({year1}.*?)</a></th>', resp.text)
days2 = re.findall(f'html">({year2}.*?)</a></th>', resp.text)
days = days1 + days2

# Remove duplicates and convert strings to datetime
days = remove_duplicates(days)
dates = [dt.datetime.strptime(d, '%Y-%m-%d') for d in days]

print(dates[:10])

[datetime.datetime(2018, 10, 3, 0, 0), datetime.datetime(2018, 10, 4, 0, 0), datetime.datetime(2018, 10, 5, 0, 0), datetime.datetime(2018, 10, 6, 0, 0), datetime.datetime(2018, 10, 7, 0, 0), datetime.datetime(2018, 10, 8, 0, 0), datetime.datetime(2018, 10, 9, 0, 0), datetime.datetime(2018, 10, 10, 0, 0), datetime.datetime(2018, 10, 11, 0, 0), datetime.datetime(2018, 10, 13, 0, 0)]


In [27]:
print(f'Number of days NHL regular season played in {year2}: ', len(dates))

Number of days NHL regular season played in 2019:  223


Now we will write a function that will scrape covers.com for the sportsbook data we need. It will take a date and return a list of dictionaries. Each dictionary item represents 1 game played on that date. The data that will be stored in the dictionaries will be as follows:

- date
- a numeric game id for that game
- home team
- away team
- home money line in american odds (ex. -110 means if you need to bet 110 to win 100 while +120 means if you bet 100 you win 120)
- goals scored by the home team
- goals scored by the away team

Lets first try retrieving the required data for one day before we try everyday of the 2018/2019 season.

In [30]:
#date we will retrieve data for
date = dates[0]
# initialize empty list that will hold our data
games=[]

# get response from covers.com
date = date.strftime('%Y-%m-%d')
url = f'https://www.covers.com/sports/nhl/matchups?selectedDate={date}'
resp = requests.get(url)

# parse into BeautifulSoup
scraped_games = BeautifulSoup(resp.text, features='html.parser').findAll('div', {'class': 'cmg_matchup_game_box'})
print(scraped_games)

[<div class="cmg_matchup_game_box cmg_game_data" data-away-team-city-search="Montreal" data-away-team-fullname-search="Montreal" data-away-team-nickname-search="Canadiens" data-away-team-shortname-search="MON" data-conference="Eastern Conference" data-event-id="121530" data-following="false" data-game-date="2018-10-03 19:00:00" data-game-odd="-240" data-home-team-city-search="Toronto" data-home-team-fullname-search="Toronto" data-home-team-nickname-search="Maple Leafs" data-home-team-shortname-search="TOR" data-index="0" data-last-update="2021-12-14T23:16:26.0000000" data-league="NHL" data-link="/sport/hockey/nhl/matchup/24322" data-sdi-event-id="/sport/hockey/competition:121530" data-top-twenty-five="False">
<div class="cmg_matchup_header">
<div class="cmg_matchup_header_grabber">
<span class="cmg_follow_link covers-scoreboardFollowUTM-121530" data-event-id="121530" data-follow="true" data-league="NHL" title="Click to follow this game"></span>
</div>
<div class="cmg_matchup_header_tea

Its hard to tell from the above output but our scraped_games variable is actually a list. Each item in the list represents a gamebox from covers.com. Lets examine the first element in this list.

In [33]:
scraped_games[0]

<div class="cmg_matchup_game_box cmg_game_data" data-away-team-city-search="Montreal" data-away-team-fullname-search="Montreal" data-away-team-nickname-search="Canadiens" data-away-team-shortname-search="MON" data-conference="Eastern Conference" data-event-id="121530" data-following="false" data-game-date="2018-10-03 19:00:00" data-game-odd="-240" data-home-team-city-search="Toronto" data-home-team-fullname-search="Toronto" data-home-team-nickname-search="Maple Leafs" data-home-team-shortname-search="TOR" data-index="0" data-last-update="2021-12-14T23:16:26.0000000" data-league="NHL" data-link="/sport/hockey/nhl/matchup/24322" data-sdi-event-id="/sport/hockey/competition:121530" data-top-twenty-five="False">
<div class="cmg_matchup_header">
<div class="cmg_matchup_header_grabber">
<span class="cmg_follow_link covers-scoreboardFollowUTM-121530" data-event-id="121530" data-follow="true" data-league="NHL" title="Click to follow this game"></span>
</div>
<div class="cmg_matchup_header_team

We can now retrieve our required data

In [34]:
# iterate through all the game boxes and retrieve required information
for g in scraped_games:
    game_id = g['data-event-id'] # game_id
    h_abv = g['data-home-team-shortname-search'] # home_team
    a_abv = g['data-away-team-shortname-search'] # away_team
    h_ml = g['data-game-odd'] # home moneyline

    try:
        h_score = g.find('div', {'class': 'cmg_matchup_list_score_home'}).get_text(strip=True) # home score
        a_score = g.find('div', {'class': 'cmg_matchup_list_score_away'}).get_text(strip=True) # away score
    except:  # If a score cannot be found leave as blank
        h_score = ''
        a_score = ''

    game = {'date':date, 'game_id':game_id, 'home_team':h_abv,
            'away_team':a_abv, 'home_ml':h_ml, 'home_score':h_score,
            'away_score':a_score}

    games.append(game)
print(games)

[{'date': '2018-10-03', 'game_id': '121530', 'home_team': 'TOR', 'away_team': 'MON', 'home_ml': '-240', 'home_score': '3', 'away_score': '2'}, {'date': '2018-10-03', 'game_id': '121531', 'home_team': 'WAS', 'away_team': 'BOS', 'home_ml': '-115', 'home_score': '7', 'away_score': '0'}, {'date': '2018-10-03', 'game_id': '121532', 'home_team': 'VAN', 'away_team': 'CAL', 'home_ml': '115', 'home_score': '5', 'away_score': '2'}, {'date': '2018-10-03', 'game_id': '121533', 'home_team': 'SJ', 'away_team': 'ANA', 'home_ml': '-185', 'home_score': '2', 'away_score': '5'}]


Great, that was for 1 day. Lets throw the above code into a function and iterate to retrieve data for every game in the 2018/2019 season.

In [10]:
def nhl_games_date(date):
    """
    creates a list of NhlGame dictionaries for all games played on the date provided
    ...
    Parameters
    ----------
    date: dt.datetime
        datetime object for which we want to
    Returns
    -------
    list[NhlGame]
        a list of NhlGame objects
    """
    games = []

    # retrieve the covers.com webpage for the date provided
    date = date.strftime('%Y-%m-%d')
    url = f'https://www.covers.com/sports/nhl/matchups?selectedDate={date}'
    resp = requests.get(url)

    # parse the page, and retrieve all the game boxes on the page
    scraped_games = BeautifulSoup(resp.text, features='html.parser').findAll('div', {'class': 'cmg_matchup_game_box'})

    # iterate through all the game boxes and retrieve required information for NhlGame object
    for g in scraped_games:
        game_id = g['data-event-id'] # game_id
        h_abv = g['data-home-team-shortname-search'] # home_team
        a_abv = g['data-away-team-shortname-search'] # away_team
        h_ml = g['data-game-odd'] # home moneyline

        try:
            h_score = g.find('div', {'class': 'cmg_matchup_list_score_home'}).get_text(strip=True) # home score
            a_score = g.find('div', {'class': 'cmg_matchup_list_score_away'}).get_text(strip=True) # away score
        except:  # If a score cannot be found leave as blank
            h_score = ''
            a_score = ''

        game = {'date':date, 'game_id':game_id, 'home_team':h_abv,
                'away_team':a_abv, 'home_ml':h_ml, 'home_score':h_score,
                'away_score':a_score}

        games.append(game)

    return games

now we will use our "dates" list and the above function in order to pull data for all dates in our dates list

In [13]:
# intialize list to hold our data
games = []

# run a loop to get data for every date in dates
for date in dates:
    games += nhl_games_date(date)

# check length of games
print('There are '+ str(len(games)) + ' games in our list\n')

# print an entry
print(games[0])

There are 1358in our list

[{'date': '2018-10-03', 'game_id': '121530', 'home_team': 'TOR', 'away_team': 'MON', 'home_ml': '-240', 'home_score': '3', 'away_score': '2'}, {'date': '2018-10-03', 'game_id': '121531', 'home_team': 'WAS', 'away_team': 'BOS', 'home_ml': '-115', 'home_score': '7', 'away_score': '0'}, {'date': '2018-10-03', 'game_id': '121532', 'home_team': 'VAN', 'away_team': 'CAL', 'home_ml': '115', 'home_score': '5', 'away_score': '2'}, {'date': '2018-10-03', 'game_id': '121533', 'home_team': 'SJ', 'away_team': 'ANA', 'home_ml': '-185', 'home_score': '2', 'away_score': '5'}, {'date': '2018-10-04', 'game_id': '121536', 'home_team': 'PIT', 'away_team': 'WAS', 'home_ml': '-165', 'home_score': '7', 'away_score': '6'}, {'date': '2018-10-04', 'game_id': '121537', 'home_team': 'CAR', 'away_team': 'NYI', 'home_ml': '-170', 'home_score': '1', 'away_score': '2'}, {'date': '2018-10-04', 'game_id': '121534', 'home_team': 'BUF', 'away_team': 'BOS', 'home_ml': '115', 'home_score': '0', '

Great! We now have a database of every game in the 2018/2019 season telling us the betting line and score.

## Betting Lines and Expected Value
To understand the calibration curve we are about to create we first need to understand some betting principles.

### Coin Flipping Odds
Lets say a friend offers you a bet. He flips a coin, if it lands on heads you pay him 10 dollars if it lands on tails he will pay you 10 dollars (even odds). Should you take this bet? You will not be surprised to learn that the answer to this is it does not matter. Since there is an even chance of the coin landing on heads or tails and you friend is offering you even odds this is a 0 expected value (EV) bet. You will lose some bets you will win some bets but over many bets you will come out even.

Scenario 2. Lets say this time your friend says if it lands on heads I will give you 15 dollars and if it lands on tails you only have to give me 10. You are betting 10 to potentially win 15 which would be equivalent to a +150 moneyline bet. Should you take this bet? The obvious answer is yes. Since there is a 50% chance of the coin landing on heads but your friend is offering you greater than 50% odds this is a positive EV bet. You will lose some bets you will win some bets but over many bets you will make money.

Scenario 3. This time your friend says if it lands on heads I will give you 10 dollars if it lands on tails you owe me 15. You are betting 15 to potentially win 10 which would be equivalent to a -150 moenyline bet. Should you take this bet? The obvious answer is no. Since there is a 50% chance of the coin landing on heads but your friend is offering you less than 50% odds this is a negative EV bet. You will lose some bets you will win some bets but over many bets you will lose money

### Sports Betting
So how does this apply to sports betting? Sports betting is betting on a coin flip however the coin is not evenly weighted. Your job as a bettor is to determine the true odds and which side is the positive EV bet.

Imagine the Flames are playing the Oilers tonight. The sportsbook is offering the Flames at -200 and the Oilers at +190 which side do you bet? The process to determine this is as follows:
1. Convert the money line odds to percentages
    -200 odds is equivalent to the sportsbooks saying they think the Flames have a 66% chance of winning
    +190 odds is equivalent to the sportsbooks saying they think the Oilers have a 33% chance of winning
2. Run your own analysis to determine what percent chance you think each team has of winning. Typically you would use some sort of model to determine this. We are going to build a machine learning model that will calculate these percentages for us. Lets say you run your model and come up with the following
    Flames will win 60% of the time
    Oilers will win 40% of the time
3. Now which side do you bet? The sportsbook thinks Flames win 66% you think they win only 60% therefore betting the Flames would be negative EV. The sportsbook thinks the Oilers win 33% of the time however you think they will actually win 40% of the time therefore betting the Oilers is positive EV.

Even though the Flames are more likely to win the game betting them is negative EV and you would lose money over time.

The way sportsbooks make money is that they try to set the betting line so both sides of the bet are negative EV. This is due to a comission they bake into the line. If the book runs their model and determines that the correct line for the Flames is -200 they will actually offer the bet at say -220 this they can make both sides of the bet negative EV.

## Calibration Curve
Now that we have some betting theory out of the way lets start creating a calibration curve. This first function will take our game database list and calculate the sportsbooks accuracy in predicting games. If the predicted favourite actually ends up winning the game then that is counted as a correct prediction. If the underdog wins that is an incorrect prediction.

In [14]:
def sportsbook_accuracy(game_data):
    """
    calculates the accuracy score of the sportsbooks for the provided games
    ...
    Parameters
    ----------
    game_data: List[Dict]
        list of NhlGame objects for which we want to calculate the accuracy score for
    Returns
    -------
    accuracy: float
        the accuracy of the sportsbooks predictions
    outcomes: List[Boolean]
        True if the home team wins
    predictions: List[Boolean]
        True if the sportsbook had the home team favoured
    probabilities: List[float]
        the implied probability of the home team winning based on the moneyline odds
    """
    outcomes = []  # The actual outcome of the game. True if the home team wins
    predictions = []  # The sportsbook's "prediction". True if the home team was favoured.
    probabilities = []  # The implied probabilities determined from the moneyline odds

    for d in game_data:
        moneyline = int(d['home_ml'])
        home_score = int(d['home_score'])
        away_score = int(d['away_score'])

        if moneyline == 100:
            # We will exclude tossups for the calibration curve
            continue

        # Convert moneyline odds to their implied probabilities
        if moneyline < 0:
            probabilities.append(moneyline / (moneyline - 100))
        elif moneyline > 100:
            probabilities.append(100 / (moneyline + 100))

        outcomes.append(home_score > away_score)
        predictions.append(moneyline < 0)

    accuracy = 100 * accuracy_score(outcomes, predictions)

    return accuracy, outcomes, predictions, probabilities

In [17]:
accuracy, outcomes, predictions, probabilities = sportsbook_accuracy(games)
print('Accuracy')
print('Sportsbooks accuracy is ' + str(accuracy))
print('Outcomes')
print(outcomes[:10])
print('Predictions')
print(predictions[:10])
print('Probabilities')
print(probabilities[:10])

Accuracy
Sportsbooks accuracy is 57.564296520423596
Outcomes
[True, True, True, False, True, False, False, False, False, True]
Predictions
[True, True, False, True, True, True, False, False, False, True]
Probabilities
[0.7058823529411765, 0.5348837209302325, 0.46511627906976744, 0.6491228070175439, 0.6226415094339622, 0.6296296296296297, 0.46511627906976744, 0.39215686274509803, 0.46511627906976744, 0.6428571428571429]


We can see from the above that the sportbook correctly predicted the winner of the game 57.56% of the time